## Callin Switzer
## Run Simulations to compare output to NNET

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
import pandas as pd
import multiProcTraj # note that this is a custom-written file 
from multiprocessing import Pool, cpu_count


# define directories
baseDir = os.getcwd()
dataDir = r'D:\MothSimulations\11c-AggressiveManeuver\Qstore\hws_am_con'
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\DataOutput'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
randomRawData = r'D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/PythonGeneratedData'

if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
if not os.path.exists(savedModels):
    os.mkdir(savedModels)

print(sys.version)
now = datetime.now()
print("last run on " + str(now))

3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 11:48:23) [MSC v.1900 64 bit (AMD64)]
last run on 2019-01-02 12:12:08.162075


In [2]:
# Load data
testDF = pd.read_csv(os.path.join(dataOutput, "NNpreds_RandomICs.csv"))
testDF["x_0"] = 0 
testDF["y_0"] = 0
testDF_sm= pd.read_csv(os.path.join(dataOutput, "NNpreds_small_RandomICs.csv"))

In [3]:
print(testDF_sm.shape)
testDF.head()

(1000, 19)


,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,F_pred,alpha_pred,tau_pred,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred,x_0,y_0
0,0.450877,6.272284,541.507416,502.568101,14.035800,-17.699701,12289.356,3.978259,50358.145,9.347785,7.688051,0.654225,6.544707,385.62134,308.52344,16.010592,19.736385,0,0
1,1.913727,2.831231,1409.737698,995.727369,17.112298,19.846242,32885.816,5.655428,-43891.664,23.603876,23.633075,2.267903,3.124010,950.20660,1405.04260,16.328537,12.697647,0,0
2,1.081307,3.740158,880.949432,178.694952,-1.641278,-20.840242,31390.416,5.920581,-60930.320,11.786005,2.540601,0.827989,3.401310,314.93234,81.49345,-21.342203,-26.094006,0,0
3,4.572172,3.084163,-1401.856110,-1363.320818,-20.266189,5.480075,35029.824,0.716195,17886.117,-33.335655,-30.535096,4.439168,2.988790,-1989.83980,-1614.85940,2.205544,2.750597,0,0
4,5.882793,4.596070,1420.324857,545.096025,-3.849331,-14.273627,20258.309,3.893479,-37054.140,26.205825,13.510248,5.767939,4.434904,1218.60990,845.32600,-6.078696,-8.131236,0,0


In [4]:
testDF[["x_0", "x_dot_0", "y_0", "y_dot_0", 
           "theta_0", "theta_dot_0", "phi_0", "phi_dot_0", 
           "F_pred", "alpha_pred", "tau_pred"]].head()

,x_0,x_dot_0,y_0,y_dot_0,theta_0,theta_dot_0,phi_0,phi_dot_0,F_pred,alpha_pred,tau_pred
0,0,541.507416,0,502.568101,6.272284,-17.699701,0.450877,14.035800,12289.356,3.978259,50358.145
1,0,1409.737698,0,995.727369,2.831231,19.846242,1.913727,17.112298,32885.816,5.655428,-43891.664
2,0,880.949432,0,178.694952,3.740158,-20.840242,1.081307,-1.641278,31390.416,5.920581,-60930.320
3,0,-1401.856110,0,-1363.320818,3.084163,5.480075,4.572172,-20.266189,35029.824,0.716195,17886.117
4,0,1420.324857,0,545.096025,4.596070,-14.273627,5.882793,-3.849331,20258.309,3.893479,-37054.140


In [5]:
testDF.shape[0]


1000000

___

# Loop Version (Not used -- see parallel version below)

In [ ]:
# put data into correct order
# x,xd,y,yd,
# theta,thetad,phi,phid, 
# F, alpha, tau0


tic = time.time()
t = np.linspace(0, 0.02, num = 100, endpoint = False) # time cut into 100 timesteps
nrun = testDF.head(1000).shape[0]  #number of trajectories.

# pre-allocate arrays to fill
nstep = 100
zeroMatrix = np.zeros([nrun, nstep])
x,      xd,    y,   yd, \
theta, thetad, phi, phid = [zeroMatrix.copy() for ii in 
                                range(len([ "x",     "xd",     "y", "yd", 
                                            "theta", "thetad", "phi", "phid"]))]

# generate random initial conditions for state 0
state0 = np.array(testDF[["x_dot_0",  "y_dot_0", 
           "theta_0", "theta_dot_0", "phi_0", "phi_dot_0", 
           "F_pred", "alpha_pred", "tau_pred"]])

# loop through all the runs
for i in range(0,nrun):
    # run ODE
    state = odeint(multiProcTraj.FlyTheBug, state0[i, :], t)
    x[i,:], xd[i,:] = state[:,0], state[:,1]
    y[i,:], yd[i, :] = state[:,2], state[:,3]
    theta[i,:], thetad[i, :] = state[:,4],state[:,5]
    phi[i,:], phid[i, :] = state[:, 6], state[:,7]
    
    if(np.mod(i, 1000) == 0):
        print(i)
        print('elapsed time = ',time.time()-tic)
print('elapsed time = ',time.time()-tic)

In [ ]:
x.shape

In [ ]:
state0.shape

In [ ]:
plt.figure()
for i in range(0,nrun):
    plt.plot(x[i, :],y[i, :], label = 'trajectory x vs y')

In [ ]:
# get final states
x_f_sim = x[:, -1]
xd_f_sim = xd[:, -1]

y_f_sim = y[:, -1]
yd_f_sim = yd[:, -1]

theta_f_sim = theta[:, -1]
thetad_f_sim = thetad[:, -1]

phi_f_sim = phi[:, -1]
phid_f_sim = phid[:, -1]

In [ ]:
combDF = testDF.head(100000).copy()

In [ ]:
combDF = combDF.assign(x_99_sim = x_f_sim, 
                      y_99_sim = y_f_sim, 
                      theta_99_sim = theta_f_sim, 
                      phi_99_sim = phi_f_sim, 
                      xd_99_sim = xd_f_sim, 
                      yd_99_sim = yd_f_sim, 
                      thetad_99_sim = thetad_f_sim, 
                      phid_99_sim = phid_f_sim) 

In [ ]:
combDF.head()

In [ ]:
combDF.to_csv(os.path.join(dataOutput, "SimVsNNet_Locations_100K.csv"), index = False)

___

# parallel version

The data is read in in multiProcTraj.py

In [17]:
# put data into correct order
# x,xd,y,yd,
# theta,thetad,phi,phid, 
# F, alpha, tau0

nrun = 1000000

tic = time.time()
# loop through all the runs

p = Pool(cpu_count()-3)
stt = time.time()   
bb = p.map(multiProcTraj.flyBug_firstLast_test, range(nrun) ) 
print(time.time() -stt)

p.close()
p.join()
print(time.time() -stt)
newDF = pd.DataFrame(bb, columns= ["x0", "x_99_sim", "xd0", "xd_99_sim", "y0", "y_99_sim", "yd0", "yd_99_sim", "theta0", \
        "theta_99_sim", "thetad0", "thetad_99_sim", "phi0", "phi_99_sim", "phid0", "phid_99_sim", "F", "alpha", "tau0"])
print('elapsed time = ',time.time()-tic)

4407.431406497955
4407.735118865967
elapsed time =  4411.009620189667


In [18]:
print(newDF.shape)

newDF.head()

(1000000, 19)


,x0,x_99_sim,xd0,xd_99_sim,y0,y_99_sim,yd0,yd_99_sim,theta0,theta_99_sim,thetad0,thetad_99_sim,phi0,phi_99_sim,phid0,phid_99_sim,F,alpha,tau0
0,0.0,9.379389,541.507416,420.184887,0.0,7.392332,502.568101,257.376476,6.272284,6.568415,-17.699701,21.180879,0.450877,0.676655,14.035800,17.518998,12289.356,3.978259,50358.145
1,0.0,23.529992,1409.737698,938.944959,0.0,23.770529,995.727369,1376.972395,2.831231,3.128542,19.846242,13.824592,1.913727,2.271462,17.112298,16.926731,32885.816,5.655428,-43891.664
2,0.0,11.582655,880.949432,288.327905,0.0,2.314977,178.694952,68.413969,3.740158,3.395991,-20.840242,-25.097394,1.081307,0.822345,-1.641278,-20.769361,31390.416,5.920581,-60930.320
3,0.0,-33.084097,-1401.856110,-1945.563196,0.0,-30.962052,-1363.320818,-1738.027627,3.084163,2.948369,5.480075,-0.792618,4.572172,4.412808,-20.266189,-1.954702,35029.824,0.716195,17886.117
4,0.0,25.778578,1420.324857,1191.824819,0.0,13.682106,545.096025,839.543279,4.596070,4.446137,-14.273627,-6.729580,5.882793,5.783755,-3.849331,-4.147584,20258.309,3.893479,-37054.140


In [19]:
# align newDF with old DF
testDF.head()

,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,F_pred,alpha_pred,tau_pred,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred,x_0,y_0
0,0.450877,6.272284,541.507416,502.568101,14.035800,-17.699701,12289.356,3.978259,50358.145,9.347785,7.688051,0.654225,6.544707,385.62134,308.52344,16.010592,19.736385,0,0
1,1.913727,2.831231,1409.737698,995.727369,17.112298,19.846242,32885.816,5.655428,-43891.664,23.603876,23.633075,2.267903,3.124010,950.20660,1405.04260,16.328537,12.697647,0,0
2,1.081307,3.740158,880.949432,178.694952,-1.641278,-20.840242,31390.416,5.920581,-60930.320,11.786005,2.540601,0.827989,3.401310,314.93234,81.49345,-21.342203,-26.094006,0,0
3,4.572172,3.084163,-1401.856110,-1363.320818,-20.266189,5.480075,35029.824,0.716195,17886.117,-33.335655,-30.535096,4.439168,2.988790,-1989.83980,-1614.85940,2.205544,2.750597,0,0
4,5.882793,4.596070,1420.324857,545.096025,-3.849331,-14.273627,20258.309,3.893479,-37054.140,26.205825,13.510248,5.767939,4.434904,1218.60990,845.32600,-6.078696,-8.131236,0,0


In [20]:
combDF = newDF.merge(testDF.head(nrun), left_on=["F", "alpha", "tau0"], right_on=["F_pred", "alpha_pred", "tau_pred"])

In [21]:
print(combDF.shape)
combDF.head()

(1000000, 38)


,x0,x_99_sim,xd0,xd_99_sim,y0,y_99_sim,yd0,yd_99_sim,theta0,theta_99_sim,...,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred,x_0,y_0
0,0.0,9.379389,541.507416,420.184887,0.0,7.392332,502.568101,257.376476,6.272284,6.568415,...,9.347785,7.688051,0.654225,6.544707,385.62134,308.52344,16.010592,19.736385,0,0
1,0.0,23.529992,1409.737698,938.944959,0.0,23.770529,995.727369,1376.972395,2.831231,3.128542,...,23.603876,23.633075,2.267903,3.124010,950.20660,1405.04260,16.328537,12.697647,0,0
2,0.0,11.582655,880.949432,288.327905,0.0,2.314977,178.694952,68.413969,3.740158,3.395991,...,11.786005,2.540601,0.827989,3.401310,314.93234,81.49345,-21.342203,-26.094006,0,0
3,0.0,-33.084097,-1401.856110,-1945.563196,0.0,-30.962052,-1363.320818,-1738.027627,3.084163,2.948369,...,-33.335655,-30.535096,4.439168,2.988790,-1989.83980,-1614.85940,2.205544,2.750597,0,0
4,0.0,25.778578,1420.324857,1191.824819,0.0,13.682106,545.096025,839.543279,4.596070,4.446137,...,26.205825,13.510248,5.767939,4.434904,1218.60990,845.32600,-6.078696,-8.131236,0,0


In [22]:
# save combined DF
combDF.to_csv(os.path.join(dataOutput, "SimVsNNet_Locations_Full.csv"), index = False)